In [1]:
!pip install python-terrier

!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git


  Cloning https://github.com/cmacdonald/pyterrier_colbert.git to /tmp/pip-req-build-8e8r_zkb
  Running command git clone -q https://github.com/cmacdonald/pyterrier_colbert.git /tmp/pip-req-build-8e8r_zkb
  Created wheel for pyterrier-colbert: filename=pyterrier_colbert-0.0.1-cp37-none-any.whl size=11525 sha256=840260096cf76f8e5f4b98ee31790dee816db900abd08aa858c8a2ede06cf1d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-_o4lqp8q/wheels/d1/a6/7b/ed4b36db97028fd43d5479485951e33828df1b948d5808ce1d
Successfully built pyterrier-colbert
  Found existing installation: pyterrier-colbert 0.0.1
    Uninstalling pyterrier-colbert-0.0.1:
      Successfully uninstalled pyterrier-colbert-0.0.1


In [2]:
import sys
COLAB='google.colab' in sys.modules
try:
  import faiss
  faiss.get_num_gpus()
except:
  if COLAB:
    print('Installing faiss-gpu from pip ')
    !pip install faiss-gpu==1.6.3

  else:
    print('Installing faiss-gpu via Conda')
    !conda install -c pytorch faiss-gpu


import faiss
assert faiss.get_num_gpus() > 0


In [3]:
import os, shutil
import pyterrier as pt
pt.init()


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [4]:
checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"


In [7]:

index_root="/content/index"
index_name="index_name"
REINDEX=False

if REINDEX or not os.path.exists(os.path.join(index_root, index_name)):

  dataset = pt.get_dataset("irds:vaswani")
  import pyterrier_colbert.indexing 
  
  if os.path.exists(os.path.join(index_root, index_name)):
      shutil.rmtree(os.path.join(index_root, index_name))

  indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, index_root, index_name, chunksize=3)
  indexer.index(dataset.get_corpus_iter())

In [8]:
!du -csh /content/index/index_name/

163M	/content/index/index_name/
163M	total


In [9]:
if REINDEX:
  pytcolbert = indexer.ranking_factory()
  
else:
  from pyterrier_colbert.ranking import ColBERTFactory
  pytcolbert = ColBERTFactory(checkpoint, index_root, index_name)
colbert_e2e = pytcolbert.end_to_end()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Mar 13, 12:59:26] #> Loading model checkpoint.
[Mar 13, 12:59:26] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip


/usr/local/lib/python3.7/dist-packages/torch/hub.py:452: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Mar 13, 12:59:48] #> checkpoint['epoch'] = 0
[Mar 13, 12:59:48] #> checkpoint['batch'] = 44500
[Mar 13, 12:59:48] #> Loading the FAISS index from /content/index/index_name/ivfpq.100.faiss ..
[Mar 13, 12:59:48] #> Building the emb2pid mapping..
[Mar 13, 12:59:49] len(self.emb2pid) = 581496


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

Loading reranking index, memtype=mem


Loading index shards to memory: 100%|██████████| 1/1 [00:04<00:00,  4.12s/shard]


In [10]:
colbert_e2e.search("chemical reactions")

lookups:   0%|          | 0/1000 [00:00<?, ?d/s]

qid 1 retrieved docs 2861


lookups: 100%|██████████| 861/861 [00:00<00:00, 30588.39d/s]


,qid,query,docid,query_toks,query_embs,docno,score,rank
2024,1,chemical reactions,4911,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",4912,19.822016,0
2590,1,chemical reactions,7048,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",7049,19.053322,1
2464,1,chemical reactions,6479,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",6480,18.036819,2
571,1,chemical reactions,9373,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",9374,17.140938,3
2413,1,chemical reactions,6278,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",6279,16.792570,4
...,...,...,...,...,...,...,...,...
1892,1,chemical reactions,4421,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",4422,3.079514,2856
1190,1,chemical reactions,10613,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",10614,2.950519,2857
2485,1,chemical reactions,6572,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",6573,2.767678,2858
793,1,chemical reactions,9832,"[tensor(101), tensor(1), tensor(5072), tensor(...","[[tensor(0.0680), tensor(-0.0085), tensor(0.11...",9833,2.278373,2859


In [11]:
dataset = pt.get_dataset("vaswani")
bm25 = pt.BatchRetrieve(dataset.get_index(), wmodel="BM25")
pt.Experiment(
    [bm25, colbert_e2e],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=["map"],
    baseline=0,
    names = ["BM25", "ColBERT E2E"]
)

qid 1 retrieved docs 5945
qid 2 retrieved docs 5744
qid 3 retrieved docs 6178
qid 4 retrieved docs 4401
qid 5 retrieved docs 4499
qid 6 retrieved docs 3616
qid 7 retrieved docs 5343
qid 8 retrieved docs 5198
qid 9 retrieved docs 5666
qid 10 retrieved docs 6776
qid 11 retrieved docs 4036
qid 12 retrieved docs 5589
qid 13 retrieved docs 5048
qid 14 retrieved docs 6743
qid 15 retrieved docs 5042
qid 16 retrieved docs 6486
qid 17 retrieved docs 6001
qid 18 retrieved docs 4834
qid 19 retrieved docs 5286
qid 20 retrieved docs 5125
qid 21 retrieved docs 4293
qid 22 retrieved docs 5287
qid 23 retrieved docs 5257
qid 24 retrieved docs 4210
qid 25 retrieved docs 5481
qid 26 retrieved docs 4005
qid 27 retrieved docs 5711
qid 28 retrieved docs 5957
qid 29 retrieved docs 4472
qid 30 retrieved docs 5807
qid 31 retrieved docs 6252
qid 32 retrieved docs 4643
qid 33 retrieved docs 5546
qid 34 retrieved docs 7008
qid 35 retrieved docs 4396
qid 36 retrieved docs 4905
qid 37 retrieved docs 5119
qid 38 ret

lookups: 100%|██████████| 898/898 [00:00<00:00, 29191.00d/s]


,name,map,map +,map -,map p-value
0,BM25,0.296517,NaN,NaN,NaN
1,ColBERT E2E,0.278724,49.0,44.0,0.228385
